# Load libraries

In [56]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
import scipy.io
import os
import sys
from pathlib import Path
import pickle
from gc import collect
from glob import glob
from datetime import datetime
import traceback
from time import sleep
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import make_axes_locatable


#Important!! Make sure your current directory is the MHDTurbPy folder!
os.chdir("/Users/nokni/work/MHDTurbPy")


# Make sure to use the local spedas
sys.path.insert(0, os.path.join(os.getcwd(), 'pyspedas'))
import pyspedas
from pyspedas.utilities import time_string
from pytplot import get_data
from joblib import Parallel, delayed

""" Import manual functions """

sys.path.insert(1, os.path.join(os.getcwd(), 'functions'))
import calc_diagnostics as calc
import TurbPy as turb
import general_functions as func
import Figures as figs
from   SEA import SEA


sys.path.insert(1, os.path.join(os.getcwd(), 'functions/3d_anis_analysis_toolboox'))
import collect_wave_coeffs 
import data_analysis 



# Better figures
from matplotlib import pyplot as plt
plt.style.use(['science', 'scatter'])
plt.rcParams['text.usetex'] = True

os.environ["CDF_LIB"] = "/Applications/cdf/cdf/lib"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Download  WIND data

In [ ]:
from joblib import Parallel, delayed

# Choose PSP or SolO
sc                      = 5                                    #  Wind:5

# Choose interval type
interval_type           = 'fast_alfvenic'                     # Options   ['slow_non_alfvenic', 'slow_alfvenic', 'fast_alfvenic']

three_sec_resol         = False                               # Whether or not to donlowad full resol mag data or 3sec resol

# If you only need to download 1 interval (dates wont matter if only_one_interval=0)
only_one_interval       = 0                                    # If you only want to check one interval

cdf_lib_path            = '/Applications/cdf/cdf/lib'            # You will need to read cdf files. You have to specify the path
choose_working_dir      = '/Users/nokni/work/sc_data/'           # Working dir. Usually where you save raw data (a psp_data, and/or solar_orbiter_data folder will be created)


#User defined parameters
addit_time_around       = 4                                     # [start_time -addit_time_around [h], end_time + addit_time_around [h]]
high_resol_data         = True
subtract_rol_mean       = 1                                      # To estimate fluctuations of mag, vel field
rolling_window          = '1h'                                   # When estimating fluctuations, size of window (centered around -w/2 -w/2
gap_time_threshold      = 5                                      # Threshold for large gaps (units of seconds)
estimate_PSD            = 0                                      # Estimate magentic field powes spectral density (keep false)
estimate_PSD_V          = 0                                      # Estimate velocity field powes spectral density (keep false)
high_res_mag            = 0                                      # Use high resol or low resolution magnetic field data ( Choose either 1 or 0 respectively)
in_RTN                  = 1                                      # RTN or spacecraft frame ( We will usually use RTN)
f_min_spec              = 2e-3                                   # Integrate over [f_min_spec, f_max_spec]to estimate σc, σr 
f_max_spec              = 1e-2                                   #
step                    = '1H'                                  # Move starting point by step (in Hours)
duration                = '3H'                                  # Duration of interval (in Hours)
settings                =  {
                            'particle_mode': '9th_perih_cut',    # either: 'spc', 'span', '9th_perih_cut'
                            'use_hampel'   : False,              # Use hampelfilter to despike plasma data
                            'part_resol'   : 600,                # Max resol of plasma data                [ms]
                            'MAG_resol'    : 1                   # Max resol og magnetic field data        [ms]

                            } 

credentials         = { 'psp':{
                               'fields': {'username': None, 'password': None},
                               'sweap' : {'username': None, 'password': None}
                              }
                      }

if sc==0:
    vars_2_downnload = {'mag': None, 'span': None,'span-a': None, 'spc': None, 'qtn': None, 'ephem': None}
    
elif sc==1:
    vars_2_downnload = {'mag': None, 'qtn': None, 'swa': None, 'ephem': None} 
else:
    vars_2_downnload = {'mag': None, 'qtn': None, 'swa': None, 'ephem': None} 
    
    
# on the other hand if you have a list of intervals
load_dir_path           = '/Users/nokni/work/3d_anisotropy/wind_3d_anisotropy/'


if sc ==5:
    if interval_type =='slow_alfvenic':
        
        load_path               = str(load_dir_path)+'/selected_intervals/slow_alfvenic/all_intervals/_wind_slow_alfvenic_intervals.pkl' # In case you have multiple days you want to download ()
        
        if three_sec_resol:
            save_path               = str(load_dir_path)+'/final_intervals/slow_alfvenic_3sec/'
        else:
            save_path               = str(load_dir_path)+'/final_intervals/slow_alfvenic/'                               # This will be connected with the parent of load_path
        
    elif interval_type =='fast_alfvenic':
        
        load_path               = str(load_dir_path)+'/selected_intervals/fast_alfvenic/all_intervals/_wind_fast_alfvenic_intervals.pkl' # In case you have multiple days you want to download ()
        if three_sec_resol:
            save_path               = str(load_dir_path)+'/final_intervals/fast_alfvenic_3sec/'
        else:
            save_path               = str(load_dir_path)+'/final_intervals/fast_alfvenic/'                               # This will be connected with the parent of load_path
        
    elif interval_type =='slow_non_alfvenic':
        load_path               = str(load_dir_path)+'/selected_intervals/slow_non_alfvenic/all_intervals/_wind_slow_non_alfvenic_intervals.pkl' # In case you have multiple days you want to download ()
        
        if three_sec_resol:
            save_path               = str(load_dir_path)+'/final_intervals/slow_non_alfvenic_3sec/'
        else:
            save_path               = str(load_dir_path)+'/final_intervals/slow_non_alfvenic/'                               # This will be connected with the parent of load_path
                       
    else:
        print('No such type! Try Again!')
else:
    print('under consruction')

    

# Change to to specified working dir
os.chdir(choose_working_dir)

# load dataframe
df                      =  pd.read_pickle(load_path)

# Convert 'End' and 'Start' columns to datetime if they are not already
df['End'] = pd.to_datetime(df['End'])
df['Start'] = pd.to_datetime(df['Start'])

# Calculate 'dts'
dts = (df['End'] - df['Start']).dt.total_seconds() / 3600


# Create a mask for rows where 'dts' is greater than 240
mask = dts > 240

# Drop rows based on the mask
df_fin  = df[~mask].reset_index(drop=True)




# Define final path
final_path              =  Path(save_path)


# # Call function
Parallel(n_jobs=1)(delayed(data_analysis.download_files)(ok, 
                                                            df_fin, 
                                                            final_path, 
                                                            only_one_interval, 
                                                            step, 
                                                            duration, 
                                                            addit_time_around, 
                                                            settings, 
                                                            vars_2_downnload, 
                                                            cdf_lib_path, 
                                                            credentials, 
                                                            gap_time_threshold, 
                                                            estimate_PSD_V, 
                                                            subtract_rol_mean, 
                                                            rolling_window, 
                                                            f_min_spec, 
                                                            f_max_spec, 
                                                            estimate_PSD, 
                                                            sc, 
                                                            high_resol_data, 
                                                            in_RTN,
                                                            three_sec_resol = three_sec_resol) for ok in range(len(df_fin)))

